In [1]:
import time
import json
from openai import OpenAI
from dotenv import dotenv_values


In [2]:
env = dotenv_values()


In [3]:
client = OpenAI(api_key=env['OPENAI_API_KEY'])
assistant_id = "asst_FLAPdrar0EUZgucq1x5aXlER"


In [16]:
prompt_base = "For the user search query provided below, return a subset of documents provided above, which should be retrieved by the query. Format your response as a JSON array of strings, where each string represents the document ID of a document that should be retrieved. If you believe that no documents should be retrieved, return an empty array."

prompt_base = "For the user search query provided below, return a subset of documents provided above, which should be retrieved by the query. Format your response as a JSON array of object, where each object consists of the document ID and document title of a document that should be retrieved. If you believe that no documents should be retrieved, return an empty array."

def get_retrieval_prompt(query):
  return f"""
    {prompt_base}

    Query: "{query}"
  """


In [24]:
with open("01_summaries.json", "r") as f:
  documents = json.load(f)

  documents_str = json.dumps(
    documents,
    indent=2,
    ensure_ascii=False
  )

print(documents_str)


[
  {
    "id": "8288e199-5927-4eb1-8c40-07387287264f",
    "title": "Odporúčanie pre softvérových inžinierov",
    "summary": "Dokument sa zaoberá vývojom odporúčacích systémov orientovaných na softvér, pričom zdôrazňuje potrebu refaktoringu a zlepšovania kvality kódu. Refaktoring je proces, ktorý zlepšuje štruktúru existujúceho zdrojového kódu bez zmeny jeho vonkajšieho správania, pričom sa zameriava na odstránenie pachov v kóde, ako sú duplikovaný kód, dlhé metódy a veľké triedy. Práce analyzujú jednotlivé refaktorovacie operácie, ako vyňatie triedy, metódy, či balíka, ktoré prispievajú k vyššej čitateľnosti a zníženiu zložitosti kódu. Ďalej sa diskutuje o systémoch pre detekciu a správu duplicít v projektoch, čo je častý problém, najmä v open-source projektoch, kde je často vysoká miera duplicít. Odporúčacie systémy, ktoré navrhujú spôsoby zlepšenia procesov a znižovanie duplicít, môžu zlepšiť efektivitu tým, že používateľom ponúkajú odporúčania v podobe textových správ alebo vizua

In [6]:
with open('02_queries.json', 'r') as f:
  queries = json.load(f)

queries


['software engineering methodologies and models',
 'what are Echo State Networks in AI',
 'introduction to mathematical logic exercises',
 'principles of quantum computing explained',
 'how to improve website usability techniques',
 'basic concepts of artificial intelligence',
 'LaTeX documentation structure guide',
 'digitization of currencies and blockchain technology',
 'C programming language practical examples',
 'database systems SQL tutorial overview',
 'recommendations for software engineers best practices',
 'elements of mathematical analysis for IT students',
 'introduction to digital currencies',
 'user experience design methods for web applications',
 'learning algebra and discrete mathematics',
 'Creative Commons licenses explained',
 'project management best practices in software development',
 'artificial intelligence fundamentals and applications',
 'Blockchain use cases in modern finance',
 'importance of refactoring in software development',
 'overview of programming 

In [9]:
def get_value_from_dict(dict):
  return dict[list(dict.keys())[0]]


In [ ]:
results = {}

thread = client.beta.threads.create()

client.beta.threads.messages.create(
  thread_id=thread.id,
  content=documents_str,
  role="user"
)


Message(id='msg_ppFT21uwUXNWwIHRUknCyi09', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='[\n  {\n    "id": "8288e199-5927-4eb1-8c40-07387287264f",\n    "title": "Odporúčanie pre softvérových inžinierov",\n    "summary": "Dokument sa zaoberá vývojom odporúčacích systémov orientovaných na softvér, pričom zdôrazňuje potrebu refaktoringu a zlepšovania kvality kódu. Refaktoring je proces, ktorý zlepšuje štruktúru existujúceho zdrojového kódu bez zmeny jeho vonkajšieho správania, pričom sa zameriava na odstránenie pachov v kóde, ako sú duplikovaný kód, dlhé metódy a veľké triedy. Práce analyzujú jednotlivé refaktorovacie operácie, ako vyňatie triedy, metódy, či balíka, ktoré prispievajú k vyššej čitateľnosti a zníženiu zložitosti kódu. Ďalej sa diskutuje o systémoch pre detekciu a správu duplicít v projektoch, čo je častý problém, najmä v open-source projektoch, kde je často vysoká miera duplicít. Odporúčacie systémy, ktoré n

In [19]:
def get_retrieval_ids(query):
  client.beta.threads.messages.create(
    thread_id=thread.id,
    content=get_retrieval_prompt(query),
    role="user"
  )

  run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant_id,
  )

  if run.status != "completed":
    raise Exception("Run failed:", run.status)

  messages_cursor = client.beta.threads.messages.list(thread.id)
  messages = [message for message in messages_cursor]
  message = messages[0]

  response_content = message.content[0].text.value
  response = json.loads(response_content)

  return get_value_from_dict(response)


In [22]:
for query in queries[3:10]:
  results[query] = get_retrieval_ids(query)


In [15]:
client.beta.threads.delete(thread.id)


ThreadDeleted(id='thread_0gPuEVRxw1i00q6tBkST2Og2', deleted=True, object='thread.deleted')

In [23]:
with open('03_retrieval.json', 'w') as f:
  json.dump(results, f, indent=2, ensure_ascii=False)


In [ ]:
# Ensure that all returned relevant documents for each query are in the original document set

document_ids = set([doc['id'] for doc in documents])

for key in results.keys():
  rel_docs = results[key]
  for rel_doc in rel_docs:
    if rel_doc['id'] not in document_ids:
      print("Document not found:", rel_doc['id'], rel_doc['title'])
